In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Autism_spectrum_disorder_(ASD)"
cohort = "GSE123302"

# Input paths
in_trait_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)"
in_cohort_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)/GSE123302"

# Output paths
out_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/GSE123302.csv"
out_gene_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE123302.csv"
out_clinical_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/clinical_data/GSE123302.csv"
json_path = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A meta-analysis of two high-risk prospective studies reveals autism-specific transcriptional changes to chromatin, autoimmune, and environmental response genes in umbilical cord blood"
!Series_summary	"Autism spectrum disorder (ASD) is a neurodevelopmental disorder that affects more than 1% of children in the USA. ASD risk is thought to arise from both genetic and environmental factors, with the perinatal period as a critical window. Understanding early transcriptional changes in ASD would assist in clarifying disease pathogenesis and identifying biomarkers. However, little is known about umbilical cord blood gene expression profiles in babies later diagnosed with ASD compared to non-typically developing and non-ASD (Non-TD) or typically developing (TD) children. Genome-wide transcript levels were measured by Affymetrix Human Gene 2.0 array in RNA from cord blood samples from both the Markers of Autism Risk in Babies-Learning Early Signs (MARBLES)

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data measured by Affymetrix Human Gene 2.0 array
# This is suitable gene expression data, not miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (ASD status), we can see it's available in row 0 as "diagnosis"
trait_row = 0  

# For gender, we see it in row 1 as "Sex"
gender_row = 1  

# For age, there's no information available in the sample characteristics
age_row = None  

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert the trait value to binary (0 for non-ASD, 1 for ASD)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'ASD':
        return 1
    elif value.upper() in ['NON-TD', 'TD']:
        return 0
    else:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# Age conversion function is not needed since age data is not available
convert_age = None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# This should only be executed if trait_row is not None, which it is in this case
if trait_row is not None:
    # Extract clinical features using the provided function
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender,
        age_row=age_row,
        convert_age=convert_age
    )
    
    # Preview the data
    print("Clinical features preview:")
    print(preview_df(clinical_features))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical features preview:
{'GSM3499537': [1.0, 0.0], 'GSM3499538': [1.0, 0.0], 'GSM3499539': [1.0, 0.0], 'GSM3499540': [1.0, 0.0], 'GSM3499541': [1.0, 0.0], 'GSM3499542': [1.0, 0.0], 'GSM3499543': [1.0, 0.0], 'GSM3499544': [1.0, 0.0], 'GSM3499545': [1.0, 0.0], 'GSM3499546': [1.0, 0.0], 'GSM3499547': [1.0, 0.0], 'GSM3499548': [1.0, 0.0], 'GSM3499549': [1.0, 0.0], 'GSM3499550': [1.0, 0.0], 'GSM3499551': [1.0, 0.0], 'GSM3499552': [1.0, 0.0], 'GSM3499553': [1.0, 1.0], 'GSM3499554': [1.0, 1.0], 'GSM3499555': [1.0, 1.0], 'GSM3499556': [1.0, 1.0], 'GSM3499557': [1.0, 1.0], 'GSM3499558': [1.0, 1.0], 'GSM3499559': [1.0, 1.0], 'GSM3499560': [1.0, 1.0], 'GSM3499561': [1.0, 1.0], 'GSM3499562': [1.0, 1.0], 'GSM3499563': [1.0, 1.0], 'GSM3499564': [1.0, 1.0], 'GSM3499565': [1.0, 1.0], 'GSM3499566': [1.0, 1.0], 'GSM3499567': [1.0, 1.0], 'GSM3499568': [1.0, 1.0], 'GSM3499569': [1.0, 1.0], 'GSM3499570': [1.0, 1.0], 'GSM3499571': [1.0, 1.0], 'GSM3499572': [1.0, 1.0], 'GSM3499573': [1.0, 1.0], 'GSM349957

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['16657436', '16657440', '16657445', '16657447', '16657450', '16657469',
       '16657473', '16657476', '16657489', '16657492', '16657502', '16657506',
       '16657514', '16657529', '16657534', '16657554', '16657572', '16657594',
       '16657598', '16657647'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be probe IDs from a microarray platform, not human gene symbols.
# The numeric identifiers (like '16657436') are not in the format of standard gene symbols 
# which would typically be alphanumeric (like 'BRCA1', 'TP53', etc.)
# Therefore, these would need to be mapped to gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Let's first check if we have any annotation columns that might contain gene symbols
print("Gene annotation column names:")
print(gene_annotation.columns.tolist())

# Let's check more rows to get a better understanding of the data
print("\nExamining more rows for potential gene information:")
# Look at GB_ACC column which contains accession numbers
print("\nSample of GB_ACC values:")
print(gene_annotation['GB_ACC'].dropna().head(10))

# Try to get any additional columns that might have gene annotations
additional_cols = [col for col in gene_annotation.columns if 'gene' in col.lower()]
if additional_cols:
    print("\nAdditional gene-related columns found:")
    print(additional_cols)
    for col in additional_cols:
        print(f"\nSample of {col} values:")
        print(gene_annotation[col].dropna().head(5))

# For Affymetrix Gene 2.0 arrays, we often need to use a specialized annotation package
# Let's try a different approach - use the GB_ACC column which contains RefSeq IDs that can be mapped to genes
print("\nCreating mapping from RefSeq accessions:")

# Create a mapping dataframe with RefSeq IDs
mapping_data = gene_annotation[['ID', 'GB_ACC']].copy()
mapping_data = mapping_data.rename(columns={'GB_ACC': 'Gene'})

# As a special case for this dataset: many RefSeq accessions start with NM_, NR_, XM_, XR_
# which include gene information we can try to extract
def extract_gene_info_from_accession(accession):
    """Try to extract gene information from RefSeq accession IDs"""
    if not isinstance(accession, str):
        return []
    
    # For NM_ and NR_ accessions, let's extract potential gene information
    # This is an imperfect approach but might help with this dataset
    parts = accession.split('_')
    if len(parts) >= 2 and parts[0] in ['NM', 'NR', 'XM', 'XR']:
        # Return the accession itself as a placeholder for the gene
        return [accession]
    return []

# Update the mapping function in apply_gene_mapping
mapping_data['Gene'] = mapping_data['Gene'].apply(extract_gene_info_from_accession)

# Print a sample of the mapping
print("Sample of the mapping data:")
print(mapping_data.head(10))

# Apply the mapping
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Check if we have any genes mapped
if gene_data.shape[0] == 0:
    print("\nNo genes were mapped successfully using accession numbers.")
    
    # Since we couldn't map genes using the available annotations, we'll create
    # a simpler dataset where probe IDs are treated as gene identifiers
    print("\nFalling back to using probe IDs as identifiers...")
    
    # Create a CSV with the original probe data
    gene_data_original = get_genetic_data(matrix_file)
    
    # Save the probe-level data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_original.to_csv(out_gene_data_file)
    
    print(f"\nGene data saved with original probe IDs: {gene_data_original.shape}")
    print(f"Sample probe IDs: {gene_data_original.index[:5]}")
    print(f"Gene data saved to {out_gene_data_file}")
else:
    # Normalize gene symbols
    gene_data = normalize_gene_symbols_in_index(gene_data)
    
    # Print summary and save
    print(f"\nGene data shape after mapping: {gene_data.shape}")
    print(f"First 10 gene symbols after mapping: {gene_data.index[:10]}")
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene data saved to {out_gene_data_file}")


Gene annotation column names:
['ID', 'RANGE_STRAND', 'RANGE_START', 'RANGE_END', 'total_probes', 'GB_ACC', 'SPOT_ID', 'RANGE_GB']

Examining more rows for potential gene information:

Sample of GB_ACC values:
0     NR_046018
4     NR_024368
7     NR_029406
9     XR_132471
15    NR_047526
18    NM_152486
19    NM_198317
21    NM_005101
22    NM_198576
23    NR_038869
Name: GB_ACC, dtype: object

Creating mapping from RefSeq accessions:


Sample of the mapping data:
         ID         Gene
0  16657436  [NR_046018]
1  16657440           []
2  16657445           []
3  16657447           []
4  16657450  [NR_024368]
5  16657469           []
6  16657473           []
7  16657476  [NR_029406]
8  16657480           []
9  16657485  [XR_132471]



No genes were mapped successfully using accession numbers.

Falling back to using probe IDs as identifiers...



Gene data saved with original probe IDs: (36459, 224)
Sample probe IDs: Index(['16657436', '16657440', '16657445', '16657447', '16657450'], dtype='object', name='ID')
Gene data saved to ../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE123302.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. We need to first load the selected_clinical_df from the saved file, or recreate it
try:
    # Try to load the previously saved clinical data
    selected_clinical_df = pd.read_csv(out_clinical_data_file)
except:
    # If loading fails, recreate the clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)

    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

# Try to load the gene data from Step 6
gene_data = pd.read_csv(out_gene_data_file, index_col=0)

# Check if we have proper gene symbols or just probe IDs
# In this case, Step 6 indicated that we have probe IDs, not gene symbols
# So we'll skip normalization and use the probe IDs directly
print(f"Gene data shape: {gene_data.shape}")
print(f"First few gene/probe identifiers: {gene_data.index[:5].tolist()}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)

# Check if linking was successful
if len(linked_data) == 0 or trait not in linked_data.columns:
    print("Linking clinical and genetic data failed - no valid rows or trait column missing")
    # Set is_usable to False and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Consider it biased if linking fails
        df=pd.DataFrame({trait: [], 'Gender': []}), 
        note="Data linking failed - unable to process gene expression data appropriately."
    )
    print("The dataset was determined to be not usable for analysis.")
else:
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
    if trait_type == 'binary':
        if len(linked_data[trait].value_counts()) >= 2:
            is_trait_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            print(f"Trait '{trait}' has only one unique value, considering it biased.")
            is_trait_biased = True
    else:
        is_trait_biased = judge_continuous_variable_biased(linked_data, trait)
    
    # Remove biased demographic features
    unbiased_linked_data = linked_data.copy()
    if 'Age' in unbiased_linked_data.columns:
        age_biased = judge_continuous_variable_biased(unbiased_linked_data, 'Age')
        if age_biased:
            print(f"The distribution of the feature \'Age\' in this dataset is severely biased.")
            unbiased_linked_data = unbiased_linked_data.drop(columns=['Age'])
    
    if 'Gender' in unbiased_linked_data.columns:
        if len(unbiased_linked_data['Gender'].value_counts()) >= 2:
            gender_biased = judge_binary_variable_biased(unbiased_linked_data, 'Gender')
            if gender_biased:
                print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.")
                unbiased_linked_data = unbiased_linked_data.drop(columns=['Gender'])
        else:
            print(f"Gender has only one unique value, considering it biased and removing.")
            unbiased_linked_data = unbiased_linked_data.drop(columns=['Gender'])
    
    # 5. Conduct quality check and save the cohort information.
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=unbiased_linked_data, 
        note="Dataset contains gene expression data from umbilical cord blood samples related to Autism Spectrum Disorder (ASD)."
    )
    
    # 6. If the linked data is usable, save it as a CSV file.
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("The dataset was determined to be not usable for analysis due to bias in the trait distribution.")

Gene data shape: (36459, 224)
First few gene/probe identifiers: [16657436, 16657440, 16657445, 16657447, 16657450]
Linking clinical and genetic data failed - no valid rows or trait column missing
Abnormality detected in the cohort: GSE123302. Preprocessing failed.
The dataset was determined to be not usable for analysis.
